# ChatData

## Sqlite
We are using a file system called **sqlite**. It looks and acts like a real single user relational database (RDB). sqlite3 comes packaged with python. You do not need to install the library. You simply import it (as below).

- We are using Pandas to write and read to sqlite. Pandas will manage a lot of the complexity of dealing with a RDB. There are other ways of reading and writing to a RDB that are VERY common and often used in production systems.  

- The most common other way is to read or write to the RDB row by row. As you can see Pandas puts the entire dataframe into the RDB or extracts a new dataframe from the RDB. These are not row by row operations. These are set operations. Set operations ARE more efficient. However, it is common to use row by row operations to avoid needing large memory computers to hold the dataframes. One row only takes a few bytes. An entire dataframe could be many gigabytes and even petabytes of data. Obviously you will end up with problems with your compute resource if your files are this big. This will not happen to you in this activity and may never happen to you while you are a data analyst. Please just be aware of this. 

- This activity is primarily about querying a RDB, so, we are using a simple way that Pandas provides. If you want to learn the more complex way, just google 'reading and wrting to a relational database using python'. There are many resources to learn from.

- The data types that sqlite supports are quite limited. for example, it does not have a DATE type.  This is not a challenge for this project. However, more sophisticated database systems,such as Postgresql, have a large array of data types, such as "DATE", etc. that give those systems additional capability. 

- All of the SQL queries could also be performed on the Pandas DataFrames directly.  You may want to try this yourself for comparison (but make sure you do the SQL queries first, as this is an exercise in using SQL!).


## SQL Magic
Within the Jupyter notebook we will be using something called **SQL Magic**.  This provides a convenient way to write SQL queries directly into code cells in the notebook and to read the results back into a Pandas DataFrame.  This makes working with SQL much easier!

Note that you may need to update your sqlite version using `conda update -c anaconda sqlite` in order for the SQL Magic to work correctly.

## The Data Analysis Lifecycle
The sections in this notebook follow the stages of the Data Analysis Lifecycle introduced in an earlier activity.  The stages are:

- Acquire
- Transform
- Organise
- Analyse
- Communicate
- Maintain


The requirements document for this project is Template SQL queries.xlsx.

# Preliminary Steps: Create a Database
First let's import Sqlite and the other libraries we will need.

In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import sqlite3

### Create the Database
Now we will create the Sqlite database.  Here is some code that does this for you.  We use the `%load_ext` magic command to load the SQL Magic extension and then use `%sql` to connect to the database.

In [3]:
!pip install ipython-sql

  Obtaining dependency information for prettytable from https://files.pythonhosted.org/packages/4d/81/316b6a55a0d1f327d04cc7b0ba9d04058cb62de6c3a4d4b0df280cbe3b0b/prettytable-3.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=2.0 from https://files.pythonhosted.org/packages/5a/a6/95f984d3cfe22e0e8a71458468ad362a8869be7ec434d78adf4347602d5c/SQLAlchemy-2.0.22-cp311-cp311-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39


In [7]:
# If the db does not exist, sqlite will create it.
con = sqlite3.connect('chatdata.db')

# loads sql magic
%load_ext sql 

# connects sql magic command to the correct db
%sql sqlite:///chatdata.db 

#### Drop the `queries` table if it already exists
The queries table will be our record of the queries created to answer the questions from the requirements spreadsheet.  As we will be running this Jupyter notebook a few times, let's drop (i.e. remove) the queries table so that we start fresh each time.  Here is the code to do this.  We use the `%%sql` magic command to tell Jupyter that we are going to write SQL in the cell.


In [29]:
%%sql
DROP TABLE IF EXISTS 'queries'

 * sqlite:///chatdata.db
Done.


[]

# Task 1: Load the Data
Now we can start loading the data.  The tables will be created as we load the data.  

Note that some of these files are quite large, so make sure you have plenty of free memory!

## Lifecycle Stages: Transform and Organise
The data has already been processed into 3 clean data files ready for this project:

- queries.csv
- posts.csv
- comments.csv

We will load these files into our database.

## Lifecycle Stage: Acquire
The data can be found in the OpenClassrooms instructions for this activity.

### Load Comments Data into a `comments` table
Now we will load the data from the csv files into our sqlite database.

First we load the csv file into a Pandas dataframe:

In [8]:
comments = pd.read_csv('/Users/raj/Desktop/SQL/chatdata/comments.csv')
comments.head()


,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


Now take the comments dataframe and push the data into the Sqlite database table called 'comments':

In [9]:
# load comments into sqlite
comments.to_sql('comments', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM comments'
comments = pd.read_sql(sql, con)
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


### Load Other CSVs
Now that you have seen the code for loading in the comments.csv now do the same to read and write the posts.csv and users.csv to sqlite. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter code in the following cells. Insert as many cells as you want to do this.
    </font></div>

In [10]:
# TODO
posts = pd.read_csv('/Users/raj/Desktop/SQL/chatdata/posts.csv')
posts.head()



,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,NaN,...,NaN,NaN,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,NaN
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,NaN,...,NaN,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,NaN,NaN
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,NaN,...,NaN,NaN,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,NaN,NaN
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,NaN,...,NaN,NaN,2019-08-24 09:57:01,NaN,NaN,0,0,0,NaN,NaN
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,NaN,...,NaN,2019-08-24 12:13:46,2019-08-24 12:13:46,NaN,NaN,0,1,0,NaN,NaN


In [34]:
# TODO
posts.to_sql('posts', con, if_exists='replace', index=False)
sql = 'SELECT * FROM posts'
posts=pd.read_sql(sql,con)
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


In [11]:
# TODO
users = pd.read_csv('/Users/raj/Desktop/SQL/chatdata/users.csv')
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,NaN,NaN,NaN,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,NaN,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,NaN,NaN,NaN,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,NaN,NaN,NaN,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [12]:
# TODO
users.to_sql('users', con, if_exists='replace', index=False)
sql = 'SELECT * FROM users'
users=pd.read_sql(sql,con)
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


### **TODO**: Drop Duplicates
Look for and drop any duplicates in all 3 of the tables (if they exist). Use Pandas to do this. If you find duplicates, you will need to rewrite the table.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter code in the following cells. 
    </font></div>

In [13]:
# TODO
print(comments.shape)
print(comments.drop_duplicates(inplace=True))

(50000, 6)
None


In [14]:
# TODO
print(posts.shape)
print(posts.drop_duplicates(inplace=True))

(42234, 21)
None


In [15]:
# TODO
print(users.shape)
print(users.drop_duplicates(inplace=True))

(18412, 13)
None


## Review the Data

Spend some time reviewing the data.  Understand what data we have, think about how that data can be used to assist in the initiative of understanding how ChatData is used in the real world.  Is the data organised in a way that would lend itself to being manged in a relational database?  How would the different tables be connected?  What are the primary and foreign keys?  Would this give you a 3NF model?

Also think about security and ethics.  Is there personal data in here?  Could individuals be identified through this data?  It it ethical to use the data in this way?  You will be asked to comment on these questions later!

Use the code below to help you.

In [16]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [17]:
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [18]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [19]:
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


In [20]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [47]:
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


# Working with Sqlite and SQL Magic
In this section let's spend a little time understanding a bit more about how we can work with Sqlite within Jupyter.

Let's look at 2 ways to query the sqlite database: using SQL Magic or using Pandas.  Either way is fine for this project.

## Writing queries with SQL Magic

You will now need to write some queries to get answers to the questions in the requirements.  

For single-line queries, start the cell with `%sql` and simply enter your query:

In [21]:
# This is an example
%sql SELECT COUNT(*) FROM comments

 * sqlite:///chatdata.db
Done.


COUNT(*)
50000


For multi line sql statements use `%%sql` as follows. This tells Jupyter that *everything* in this cell should be interpreted as sql. So, NO comments other statements are allowed:

In [22]:
%%sql
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5

 * sqlite:///chatdata.db
Done.


Id,PostId,Score,Text
723182,385124,0,"@BenBolker I don't understand. The fit cannot be done for the negative $y$. So intuitively I'd think that in order to retain the relativity of the data, one'd ideally ""mirror"" or ""flip"" it to the positive axis? Would it be possible to adjust every point individually? Basically e.g. take $abs(y_i)$ instead of minimums."
723183,385124,3,"You can't add *less* than (`-min(y)`), but you could add *more*. I'm going to stop answering now sorry, because **judging what the 'best' approach is depends on much more context that we don't have** (the goal of the analysis, why you need to fit an exponential, your level of computational and statistical sophistication and that of your audience, etc. ...)"
723186,385137,0,"nice. If you felt like doing the work it would be nice to generate an image/contour plot of log-likelihood as a function of (mu, theta) and show the lines corresponding to the two `size` values ."
723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,size=size,log=TRUE)), xlim=c(2,5),ylim=c(0.2,0.5), varnames=c(""mu"",""size""), sys3d=""contour"")abline(h=3.2/7,col=""red"")abline(h=fit.what$theta,col=""blue"")` ... although doesn't look this is actually the answer - mu-hat is independent of theta-hat ... ?"
723188,385134,0,"Don't you mean ""so variance should be $\sigma^2/(n\mu^2)$"""


## Writing queries with Pandas
Another way to write queries is to use pandas:

In [23]:
sql = """
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5
        """
result = pd.read_sql(sql, con)
result

,Id,PostId,Score,Text
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you..."
2,723186,385137,0,nice. If you felt like doing the work it would...
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si..."
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^..."


# Creating Tables with Referential Integrity

When we loaded the csv files into Sqlite database tables, Sqlite created the tables for us behind the scenes. Let's inspect this a bit more.

We can see how Sqlite created the tables by querying the `sqlite_master` table, which Sqlite uses to keep track of what objects have been created in the database:

In [24]:
%%sql
select sql from sqlite_master

 * sqlite:///chatdata.db
Done.


sql
"CREATE TABLE ""posts"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT)"
"CREATE TABLE ""comments"" (""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER)"
"CREATE TABLE ""users"" (""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER)"


The above results show the `CREATE TABLE` statements that could be used by Sqlite to recreate the tables with the exact same structure.

The problem with the CREATE TABLE statements above is that they don't enforce **referential integrity**.  In other words, they don't ensure that every UserId and PostId in the comments table refers to an actual UserId and PostId in the users and posts tables.  At the moment. we can insert any old number here, and even have multiple users with the same Id!  One of the advantages of working with relational databases is that they can enforce the correct uniqueness and relationships in the data, but at the moment we are not using that feature.  So let's fix that...

First, let's drop the original tables:


In [25]:
%%sql
DROP TABLE comments;
DROP TABLE users;
DROP TABLE posts;

 * sqlite:///chatdata.db
Done.
Done.
Done.


[]

Prove that this worked by selecting the names of the tables back.  We should have no tables:

In [26]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name


In Sqlite we need to enable the enforcement of foreign key constraints:

In [27]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///chatdata.db
Done.


[]

Now recreate the users table with a **primary key constraint** by copying the CREATE TABLE statement from above and adding the NOT NULL PRIMARY KEY clause to the Id:

In [28]:
%%sql
CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);


 * sqlite:///chatdata.db
Done.


[]

Now do the same for the posts table:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [29]:

%%sql
CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);

 * sqlite:///chatdata.db
Done.


[]

Now for the comments table.  We need to add the primary key constraint on the id here as we did for users and posts, but we also need to add FOREIGN KEY constraints on the UserId and PostId. Read the documentation here and find our how to do that: https://www.sqlite.org/foreignkeys.html.  Then create the comments table with the correct constraints:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [30]:
%%sql
CREATE TABLE "comments" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostId" INTEGER,
"Score" INTEGER,
"Text" TEXT,
"CreationDate" TEXT,
"UserId" INTEGER,
FOREIGN KEY("PostId") REFERENCES posts("PostId"),
FOREIGN KEY("UserId") REFERENCES users("UserId")
);

 * sqlite:///chatdata.db
Done.


[]

Now we can re-insert the data into these constrained tables.  First users:

In [31]:
# Insert data into the new users table
users.to_sql('users', con, if_exists='append', index=False)

18412

Now posts:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [32]:
# Insert data into the new posts table
# TODO
posts.to_sql('posts', con, if_exists='append', index=False)

42234

Finally comments, which references the users and posts tables:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [33]:
# Insert data into the new comments table
# TODO
comments.to_sql('comments', con, if_exists='append', index=False)

50000

Now check that we have the 3 new table definitions in Sqlite:

In [34]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name
comments
posts
users


We now have all the data in tables in Sqlite and the tables will enforce the referential integrity.

# Example Query and Pattern for Tasks 2 and 3

As you work through the next tasks, you will need to:

1. Prepare the Sqlite query to answer the question
2. Test it
3. Insert it into the `queries` table, so we have a record of it for others.

This is the process that we want you to follow for this project while completing it. 

Let's see an example of this by answering the following question:

**Which 5 users have viewed the most times and what is the sum of those views per user?**

## Prepare the Sqlite query

First, let's write the query:

In [35]:
sql = """
SELECT Id, SUM(Views) AS TotalViews
    FROM Users
        GROUP BY Id
            ORDER BY TotalViews DESC
                LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
result

,Id,TotalViews
0,919,85180
1,4253,35119
2,805,34637
3,7290,32639
4,3277,29255


## Test the query
You can optionally prove the query worked by performing the same query in Pandas:

In [36]:
results = users.groupby(['Id']).sum().sort_values('Views', ascending = False)[:5]
results['Views']

Id
919     85180
4253    35119
805     34637
7290    32639
3277    29255
Name: Views, dtype: int64

## Insert the query into the `queries` table
Now we need to put this query into the `queries` table in sqlite. Remember we want these queries to be accessible to everybody that should have access to them. We do not want people writing and rewriting the same queires over and over again. The easiest thing to do is create a dictionary with the values and insert these into the queries table.  Note that the values are provided as lists as we are inserting a list of values (i.e. a number of rows) into the table.  In this case the number of rows is 1, so we have lists of 1 item.

So here, we have a column called 'task' with a list of values, a column called 'action' with a list of values, etc.

In [37]:
query_dict = {
              'task': ['Single Table Queries'],
              'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
              'query': [sql]
             }
query_dict

{'task': ['Single Table Queries'],
 'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
 'query': ['\nSELECT Id, SUM(Views) AS TotalViews\n    FROM Users\n        GROUP BY Id\n            ORDER BY TotalViews DESC\n                LIMIT 5\n                ']}

Now that you have the data structure (query_dict) containing the data, create a pandas dataframe that holds those values:

In [38]:
queries = pd.DataFrame(query_dict)
queries

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


Now load that pandas dataframe (queries) into the sqlite table called queries. In this case, you use append NOT replace. You will be adding to this tables as you go thru this project.

In [39]:
# load query into sqlite
queries.to_sql('queries', con, if_exists='append', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)
queries.head()

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


It is likely that as you iterate thru this notebook you will create some duplicate entries in the query table. Not a big deal. Just tell sql or pandas (much easier in pandas) to drop the duplicates! If you drop the duplicates in Pandas you will have to write the entire dataframe back to sqlite. Otherwise it is changed in memory in Pandas but NOT in sqlite on the disk drive (or SDD). Just do this drop at the end of this notebook so that you are not constantly dealing with this. 

So, to summarise, as you go through the following tasks you need to:

- answer the question in sql
- prove it in pandas (if you want to)
- put the query into the queries table

# Task 1 (continued): Insert the CREATE TABLE Statements into the `queries` Table

Now that we understand how to populate the `queries` table, let's insert the CREATE TABLE statements into it.  First let's define a function to help us insert into the queries table:


In [41]:
# Define a function that will insert into the queries table
def store_query(task, action, query):
    query_dict = {
                  'task': [task],
                  'action': [action],
                  'query': [query]
                 }

    # put query into the query_dict
    queries = pd.DataFrame(query_dict)

    # load query into sqlite
    queries.to_sql('queries', con, if_exists='append', index=False)    

Now we can specify the queries and call the above function to store them.  The first one is done for you.

In [42]:
sql = """
    CREATE TABLE "comments" (
    "Id" INTEGER,
    "PostId" INTEGER,
    "Score" INTEGER,
    "Text" TEXT,
    "CreationDate" TEXT,
    "UserId" INTEGER
    )
    """
store_query("Task 1", "Create table comments", sql)

Let's prove it works by selecting back from the queries table:

In [43]:
# Prove it works
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"


## Insert the other CREATE TABLE statements into the `queries` table.
Follow the above pattern to complete these code cells:

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [51]:
sql = """
    CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);
    """
store_query("Task 2", "Create table posts", sql)

In [52]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT);"


In [55]:

sql = """
    CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);
    """
store_query("Task 3", "Create table users", sql)

In [56]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT);"
Task 2,Create table posts,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 3,Create table users,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"


## Count the Number of Rows in Each Table
Run some queries to count the number of rows in each of the tables.  Don't forget to insert the query into the queries table.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [180]:

sql= """
select count(id)
From comments
"""
store_query("Task 1", "row count comments", sql)



In [80]:
%%sql
select count(id)
From comments

 * sqlite:///chatdata.db
Done.


count(id)
50000


In [181]:

sql= """
select count(id)
From users
"""
store_query("Task 1", "count rows users", sql)

In [83]:
%%sql
select count(id)
From users

 * sqlite:///chatdata.db
Done.


count(id)
18412


In [182]:
sql= """
select count(id)
From posts
"""
store_query("Task 1", "count rows posts", sql)

In [85]:

%%sql
select count(id)
From posts


 * sqlite:///chatdata.db
Done.


count(id)
42234


In [86]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT);"
Task 2,Create table posts,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 3,Create table users,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 4,row count comments,select count(id)From comments
Task 4,row count comments,select count(id)From comments
Task 5,count rows users,select count(id)From users
Task 5,count rows posts,select count(id)From posts


## Do some Random Checks on the Data
Let's write some queries that select 5 random rows from each table.  The queries are provided here:

    select * from Comments order by random() limit 5;
    select * from Users order by random() limit 5;
    select * from Posts order by random() limit 5;

Enter the queries into the code cells below and insert the queries into the `queries` table in the same was as you did for the CREATE TABLE statements.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cells
    </font></div>

In [90]:

%%sql 
select * from posts
order by random()
limit 5

 * sqlite:///chatdata.db
Done.


[(385331, 1, 0, 0, '2019-01-02 18:29:31', 2, 113, '<p>I am currently writing some code which performs regression and have noticed that when I calculate variance of <span class="math-container">$c\\hat ... (1025 characters truncated) ... span> is only just greater than <span class="math-container">$p$</span> (e.g. when <span class="math-container">$n=p+1,n=p+2,... $</span>etc ).</p>\n', 232845, None, 232845, None, '2019-01-02 18:47:45', '2019-01-02 18:47:45', 'Negative values for OLS variance', '<regression><variance><least-squares>', 0, 17, 0, None, None),
 (421990, 2, 0, 415265, '2019-08-13 14:08:22', 2, 0, '<p>Comments to the question indicate the interest focuses on random variables that are unbiased linear estimators.  Despite this restriction, <strong ... (6683 characters truncated) ... iner">$\\rho = (1+1^2)^{-1/2}=\\sqrt{1/2}$</span> when <span class="math-container">$n$</span> is odd, which is supported throughout the plane.</p>\n', 919, None, 0, None, None, '2019-08-13 14:08:22', None, None, 0, 0, 0, None, None),
 (394401, 1, 0, 0, '2019-02-26 02:38:16', 0, 9, '<p>Does any one know or have references for the consistency of non-linear squares estimators in Gaussian mixture model? (or the consistency of qausi mles in Gaussian mixture model). Thank you.</p>\n\n<p>I have been struggling to find a reference several days.  </p>\n', 239044, None, 0, None, None, '2019-02-26 02:38:16', 'consistency of non-linear squares estimators in Gaussian mixture model', '<normal-distribution><mixture>', 0, 0, 0, None, None),
 (422770, 1, 0, 0, '2019-08-19 11:58:55', 1, 36, '<p>One of the problems in Brockwell and Davis book about time series is to show that </p>\n\n<p>1) if \n<span class="math-container">\\begin{equation ... (633 characters truncated) ... ntainer">$\\omega \\in [-\\pi, \\pi)$</span>.</p>\n\n<p>I can prove these two results, but what is the message this question is trying to give?</p>\n', 35011, None, 35011, None, '2019-08-20 09:16:44', '2019-08-20 09:16:44', 'Intuition of the convergence of sample ACF', '<time-series><autocorrelation><convergence><signal-processing>', 1, 0, 0, None, None),
 (424427, 1, 424475, 0, '2019-08-31 08:48:39', 5, 72, '<p>In <a href="https://stats.stackexchange.com/a/103605/163114">this most upvoted CV answer on that topic</a> the "scale" parameter (aka "sigma" in S ... (969 characters truncated) ... ion about the "Scale" (Stata: "Sigma") parameter of a tobit regression? How is it calculated, and what does it tell us about the model quality?</p>\n', 163114, None, 163114, None, '2019-08-31 09:02:00', '2019-08-31 18:26:35', 'What is really the Scale or Sigma parameter of a tobit regression?', '<r><regression><tobit-regression>', 1, 0, 0, None, None)]

In [91]:
sql= """
select * from posts
order by random()
limit 5
"""
store_query("Task 6", "random rows posts", sql)

In [92]:


%%sql 
select * from comments
order by random()
limit 5

 * sqlite:///chatdata.db
Done.


Id,PostId,Score,Text,CreationDate,UserId
775118,415347,0,"@iot In classical statistics, you might estimate the parameters by [finding the $\theta$ that maximizes $p(x\,|\,\theta)$ as a function of $\theta$](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation). So people will write $L(\theta)=p(x\,|\,\theta)$ and try to compute $\hat{\theta}_{MLE}={\arg\max}\,L(\theta)$. In this case, you don't care about $p(x\,|\,\theta)$'s ""status"" as a conditional pdf over $x$. You care about its status as a real-valued function of $\theta$ that you want to maximize with respect to $\theta$. So $L(\cdot)$-style notation is a holdover from that setting.",2019-06-29 22:23:36,173437
755889,404263,1,The simple answer is that it is a very rough approximation.,2019-04-21 13:42:11,35989
732280,389935,0,"What I want to be able to say is what SD and what SE within one subject in average can be expected. But I do not think I may calculate the SD for each subject and determine the mean value out of it, may I?",2019-01-30 15:05:58,235888
736717,392372,0,You can find some ideas here: https://stats.stackexchange.com/questions/188903/intuition-on-the-kullback-leibler-kl-divergence/189758#189758,2019-02-13 21:49:48,11887
768309,411479,1,"@FransRodenburg as an aside, I lived in Tokyo's Azabu-Juban district in the late 1990's.",2019-06-04 12:27:38,35584


In [93]:
sql= """
select * from comments
order by random()
limit 5
"""
store_query("Task 7", "random rows comments", sql)

In [95]:

%%sql 
select * from users
order by random()
limit 5

 * sqlite:///chatdata.db
Done.


Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
256297,31,2019-08-13 12:57:06,Jonathan Foldager,2019-11-21 16:10:31,None,"Palo Alto, CA, USA",None,4,0,0,https://lh3.googleusercontent.com/-XdUIqdMkCWA/AAAAAAAAAAI/AAAAAAAAAAA/4252rscbv5M/photo.jpg?sz=128,13540386
236601,1,2019-02-05 17:01:29,epistudent21,2019-10-29 15:23:27,None,None,None,0,0,0,https://www.gravatar.com/avatar/1bf7bd5d7eb3bfc6531ec7b9b77d4d9a?s=128&d=identicon&r=PG&f=1,15272010
75070,111,2015-05-01 23:58:47,dustin,2019-07-04 03:34:05,None,Bangkok Thailand,None,0,12,0,https://www.gravatar.com/avatar/85bb9e2c9145ecb17430d6fdb574410f?s=128&d=identicon&r=PG&f=1,2467328
124102,1,2016-07-22 01:50:19,TJW,2019-07-02 05:53:20,None,None,None,0,0,0,https://www.gravatar.com/avatar/f6faa6a02a01bc0eb0c00aa82de89ea0?s=128&d=identicon&r=PG&f=1,2555933
267216,101,2019-11-29 16:55:12,AlexO,2019-11-29 16:55:12,None,"Toronto, Canada","<p>Full stack dev, creating analytics software in the capital markets domain.Interested in: high performance Java, machine learning, DDD.</p>",0,0,0,https://www.gravatar.com/avatar/5dffbd0bd86e324aff9d04907df88e8b?s=128&d=identicon&r=PG,2195588


In [96]:
sql= """
select * from users
order by random()
limit 5
"""
store_query("Task 8", "random rows users", sql)

In [97]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT);"
Task 2,Create table posts,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 3,Create table users,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 4,row count comments,select count(id)From comments
Task 4,row count comments,select count(id)From comments
Task 5,count rows users,select count(id)From users
Task 5,count rows posts,select count(id)From posts
Task 6,random rows posts,select * from postsorder by random()limit 5


# Task 2: Create Single Table Queries


## Lifecycle Stage: Analyze
We can now start the analysis with our single-table queries.  First we need to create a new computed column to help with one of the queries.  The code below creates a column called LEN_BODY which is the length of the BODY text:


In [98]:
%%sql
ALTER TABLE POSTS ADD COLUMN LEN_BODY INT

 * sqlite:///chatdata.db
Done.


[]

In [99]:
%%sql
UPDATE POSTS SET LEN_BODY = LENGTH(BODY)

 * sqlite:///chatdata.db
42234 rows affected.


[]

### Single Table Queries
From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter your code below.  Just keep on inserting cells as you need them.
    </font></div>

In [105]:
sql="""
select count(*) as count
from posts
where commentcount=0
"""
store_query('task2','How many posts have 0 comments?',sql)
res=pd.read_sql(sql,con)
res.head()

,count
0,21713


In [107]:
sql="""
select count(*) as count
from posts
where commentcount=1
"""
store_query('task2','How many posts have 1 comments?',sql)
res=pd.read_sql(sql,con)
res.head()

,count
0,6460


In [108]:
sql="""
select count(*) as count
from posts
where commentcount>=2
"""
store_query('task2','How many posts have 2 comments or more?',sql)
res=pd.read_sql(sql,con)
res.head()

,count
0,14061


In [109]:
sql="""
select * 
from posts
order by viewcount desc
limit 5
"""
store_query('task2','Find the 5 posts with the highest viewcount',sql)
res=pd.read_sql(sql,con)
res.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
0,388566,1,388582,0,2019-01-22 15:16:47,56,19542,"<ul>\n<li>Statement One (S1): ""One in 80 death...",228214,None,...,2019-01-22 21:40:39,2019-01-24 17:09:47,"Is it wrong to rephrase ""1 in 80 deaths is cau...",<interpretation><risk>,9,15,15,None,None,2270
1,394118,1,394128,0,2019-02-24 14:07:11,64,16317,<p>A human child at age 2 needs around 5 insta...,107213,None,...,2019-02-25 22:40:22,2019-03-03 17:37:05,Why do neural networks need so many training e...,<neural-networks><neuroscience>,12,24,38,None,None,512
2,431370,1,431397,0,2019-10-14 11:29:21,77,11723,<p>It seems very counter intuitive to many peo...,262594,None,...,2019-11-28 01:44:34,2019-11-28 01:44:34,Is there a name for the phenomenon of false po...,<probability><terminology><intuition>,8,9,18,None,None,811
3,398646,1,398653,0,2019-03-21 01:19:52,61,9850,<p>The title of the Comment in Nature <a href=...,163067,None,...,2019-03-22 22:14:04,2019-03-30 19:35:27,"What does ""Scientists rise up against statisti...",<statistical-significance><p-value><bias>,10,7,34,None,None,2148
4,434128,1,434579,0,2019-11-01 13:07:36,73,6718,<p>I am designing a one year program in data a...,14188,None,...,None,2019-11-26 00:59:15,Famous statistical wins and horror stories for...,<mathematical-statistics><data-visualization><...,13,7,70,None,2019-11-01 15:12:41,1172


In [111]:
sql="""
select * 
from posts
order by score desc
limit 5
"""
store_query('task2','Find the top 5 posts with the highest scores',sql)
res=pd.read_sql(sql,con)
res.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,LEN_BODY
0,431397,2,0,431370,2019-10-14 14:29:36,101,0,<p>Yes there is. Generally it is termed <stron...,142976,None,...,None,2019-10-14 14:29:36,None,None,0,0,0,None,None,269
1,394128,2,0,394118,2019-02-24 15:44:44,100,0,<p>I caution against expecting strong resembla...,22311,None,...,2019-03-03 17:37:05,2019-03-03 17:37:05,None,None,0,15,0,None,None,5829
2,426878,2,0,426873,2019-09-11 23:23:31,93,0,<p><strong>tl;dr</strong> Even though this is ...,119015,None,...,2019-09-13 14:02:28,2019-09-13 14:02:28,None,None,0,6,0,None,None,4627
3,388578,2,0,388566,2019-01-22 15:48:47,80,0,"<p>To me ""1 in 80 deaths..."" is by far the cle...",227039,None,...,None,2019-01-22 15:48:47,None,None,0,11,0,None,None,572
4,431370,1,431397,0,2019-10-14 11:29:21,77,11723,<p>It seems very counter intuitive to many peo...,262594,None,...,2019-11-28 01:44:34,2019-11-28 01:44:34,Is there a name for the phenomenon of false po...,<probability><terminology><intuition>,8,9,18,None,None,811


In [112]:
sql="""
select count(*) as count, score 
from posts
group by score 
order by count desc
limit 5
"""
store_query('task2','What are the 5 most frequent scores on posts?',sql)
res=pd.read_sql(sql,con)
res.head()

,count,Score
0,19888,0
1,11867,1
2,5094,2
3,2228,3
4,1059,4


In [115]:
sql="""
select count(*) as count
from posts
where tags like '%data%'
"""
store_query('task2','How many posts have the keyword "data" in their tags?',sql)
res=pd.read_sql(sql,con)
res.head()

,count
0,2242


In [116]:
sql="""
select count(*) as count, commentcount
from posts
group by commentcount
order by count desc
"""
store_query('task2','What are the 5 most frequent commentcount for posts?',sql)
res=pd.read_sql(sql,con)
res.head()

,count,CommentCount
0,21713,0
1,6460,1
2,4966,2
3,3063,3
4,2026,4


In [117]:
sql="""
select count(*) as count
from posts
where AcceptedAnswerId>=1
"""
store_query('task2','How many posts have an accepted answer?',sql)
res=pd.read_sql(sql,con)
res.head()

,count
0,5341


In [119]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [121]:
sql="""
select avg(Reputation) as Reputation
from users
"""
store_query('task2','What is the average reputation of table users?',sql)
res=pd.read_sql(sql,con)
res.head()

,Reputation
0,312.350912


In [122]:
sql="""
select min(Reputation) as MinReputation, max(Reputation) as MaxReputation
from users
"""
store_query('task2','What are the min and max reputation of users?',sql)
res=pd.read_sql(sql,con)
res.head()

,MinReputation,MaxReputation
0,1,228662


In [123]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [126]:
sql="""
select LEN_BODY
from posts
order by ViewCount desc
limit 5
"""
store_query('task2','What is the length of the body of 5 most viewed posts?',sql)
res=pd.read_sql(sql,con)
res.head()

,LEN_BODY
0,2270
1,512
2,811
3,2148
4,1172


In [127]:
sql="""
select Count(Distinct(Location)) as count
from users

"""
store_query('task2','How many different locations are there in the users table?',sql)
res=pd.read_sql(sql,con)
res.head()

,count
0,1900


In [135]:
sql="""
select count(*) as count, Location
from users
where Location is not null
group by Location
order by count desc
limit 5
"""
store_query('task2','What are the top 5 locations of users?',sql)
res=pd.read_sql(sql,con)
res.head()

,count,Location
0,117,Germany
1,100,India
2,69,United States
3,66,"Paris, France"
4,63,"London, United Kingdom"


In [140]:
sql="""
select case cast(strftime('%w',CreationDate) as integer)
when 0 then 'Sunday'
When 1 then 'Monday'
When 2 then 'Tuesday'
When 3 then 'Wednesday'
When 4 then 'Thursday'
When 5 then 'Friday'
else 'Saturday'
end as weekday,

round(sum(ViewCount*100.0)/(Select sum(ViewCount) from posts),2) as percentage
from posts
group by weekday
order by percentage desc
"""
store_query('task2','Rank the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage.',sql)
res=pd.read_sql(sql,con)
res.head()

,weekday,percentage
0,Thursday,16.82
1,Wednesday,16.77
2,Tuesday,16.26
3,Monday,15.81
4,Friday,13.56


# Task 3: Cross Table Queries

## Lifecycle Stage: Analyze
Let's continue the analysis with our multi-table queries.  

### Cross Table Queries

From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Enter your code below.  Just keep on inserting cells as you need them.
    </font></div>



In [141]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [144]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [183]:
sql="""
select count(*)
from posts
join users on posts.OwnerUserId=users.id
where AboutMe is not null
"""
store_query('task3','How many posts have been created by a user that has a filled out the "AboutMe" section?',sql)
res=pd.read_sql(sql,con)
res.head()

,count(*)
0,17189


In [184]:
# TODO
sql="""
select count(*)/(select cast(count(*) as float) from users where AboutMe is not null) as Userposts
from posts
join users on posts.OwnerUserId=users.id
Where users.AboutMe is not null
"""
store_query('task3','Considering only the users with an "AboutMe," how many posts are there per user?',sql)
res=pd.read_sql(sql,con)
res.head()

,Userposts
0,4.274807


In [152]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [185]:
# TODO
sql="""
select count(*) as count, posts.id
from posts
join comments on posts.id=comments.PostId
group by posts.id
order by count desc
"""
store_query('task3','Not taking into account the commentcount field in the table posts, what are the Top 10 posts in terms of number of comments?',sql)
res=pd.read_sql(sql,con)
res.head()

,count,Id
0,66,386853
1,34,386556
2,31,418910
3,31,395232
4,27,402987


In [186]:
# TODO

sql="""
select count(*) as count,(posts.Score + sum(comments.Score)) as commulativescore, posts.id
from posts
join comments on posts.id=comments.PostId
group by posts.id
order by commulativescore desc
limit 10

"""
store_query('task3','What are the Top 10 posts which have the highest cummulative (post score + comment score) score? ',sql)
res=pd.read_sql(sql,con)
res.head()

,count,commulativescore,Id
0,24,306,394118
1,15,169,394128
2,11,141,388578
3,15,111,398653
4,15,101,388566


In [166]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [165]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [187]:
# TODO

sql="""
select comments.UserId, count(*) as count, users.Reputation
from users
join comments on users.id=comments.UserId
group by comments.UserId, users.Reputation
order by users.Reputation desc

"""
store_query('task3','Who are the top 10 users who comment the most? ',sql)
res=pd.read_sql(sql,con)
res.head()

,UserId,count,Reputation
0,805,1153,228662
1,919,3301,223056
2,7290,370,115531
3,686,349,85077
4,28666,158,75024


In [176]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [188]:
# TODO

sql="""
select posts.OwnerUserId, count(*) as count, users.Reputation
from posts
join users on users.id=posts.OwnerUserId

group by posts.OwnerUserId, users.Reputation
order by users.Reputation desc

"""
store_query('task3','Who are the top 10 users who post the most?',sql)
res=pd.read_sql(sql,con)
res.head()

,OwnerUserId,count,Reputation
0,805,230,228662
1,919,203,223056
2,7290,35,115531
3,686,386,85077
4,28666,8,75024


# Task 4: Check the Queries Table

Now let's tidy up and check the queries table.

First let's check it's contents:

In [189]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT);"
Task 2,Create table posts,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 3,Create table users,"CREATE TABLE ""users"" (""Id"" INTEGER NOT NULL PRIMARY KEY,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER);"
Task 4,row count comments,select count(id)From comments
Task 4,row count comments,select count(id)From comments
Task 5,count rows users,select count(id)From users
Task 5,count rows posts,select count(id)From posts
Task 6,random rows posts,select * from postsorder by random()limit 5


## Drop Duplicates

You likely have some duplicates. Lets drop them. 

In [191]:
# Read the queries table into pandas
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)

# Drop duplicates
queries.drop_duplicates(inplace = True) # drop duplicates

## Case Issues

Remember, SQL is case insensitive. Pandas IS case sensitive. Lets deal with this now by making all of the text uppercase. 

In [192]:
for col in queries.columns:
    queries[col] = queries[col].str.upper()
    
queries

,task,action,query
0,SINGLE TABLE QUERIES,WHICH 5 USERS HAVE VIEWED THE MOST TIMES AND W...,"\nSELECT ID, SUM(VIEWS) AS TOTALVIEWS\n FRO..."
1,TASK 1,CREATE TABLE COMMENTS,"\n CREATE TABLE ""COMMENTS"" (\n ""ID"" INTE..."
2,TASK 2,CREATE TABLE POSTS,"\n CREATE TABLE ""POSTS"" (\n""ID"" INTEGER NOT..."
3,TASK 2,CREATE TABLE POSTS,"\n CREATE TABLE ""USERS"" (\n""ID"" INTEGER NOT..."
4,TASK 3,CREATE TABLE USERS,"\n CREATE TABLE ""USERS"" (\n""ID"" INTEGER NOT..."
...,...,...,...
78,TASK3,"CONSIDERING ONLY THE USERS WITH AN ""ABOUTME,"" ...",\nSELECT COUNT(*)/(SELECT CAST(COUNT(*) AS FLO...
79,TASK3,NOT TAKING INTO ACCOUNT THE COMMENTCOUNT FIELD...,"\nSELECT COUNT(*) AS COUNT, POSTS.ID\nFROM POS..."
80,TASK3,WHAT ARE THE TOP 10 POSTS WHICH HAVE THE HIGHE...,"\nSELECT COUNT(*) AS COUNT,(POSTS.SCORE + SUM(..."
81,TASK3,WHO ARE THE TOP 10 USERS WHO COMMENT THE MOST?,"\nSELECT COMMENTS.USERID, COUNT(*) AS COUNT, U..."


In [193]:
# Write the now deduped uppercase dataframe back to sqlite and replace the table
queries.to_sql('queries', con, if_exists='replace', index=False)

77

## Use Case

Now that we have this queries table, lets give you some ideas about how you would use it.

Suppose you wanted to find all of the queries where you did a GROUP BY:

In [194]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%GROUP BY%'

 * sqlite:///chatdata.db
Done.


query
"SELECT ID, SUM(VIEWS) AS TOTALVIEWS FROM USERS GROUP BY ID ORDER BY TOTALVIEWS DESC LIMIT 5"
"SELECT COUNT(*) AS COUNT, SCORE FROM POSTSGROUP BY SCORE ORDER BY COUNT DESCLIMIT 5"
"SELECT COUNT(*) AS COUNT, COMMENTCOUNTFROM POSTSGROUP BY COMMENTCOUNTORDER BY COUNT DESC"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSGROUP BY LOCATIONORDER BY COUNT DESCLIMIT 5"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSGROUP BY LOCATIONORDER BY COUNT DESCWHERE LOCATION IS NOT 'NONE'LIMIT 5"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSGROUP BY LOCATIONORDER BY COUNT DESCWHERE LOCATION IS NOT NULL 'NONE'LIMIT 5"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSGROUP BY LOCATIONORDER BY COUNT DESCWHERE LOCATION IS NOT NULLLIMIT 5"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSGROUP BY LOCATIONORDER BY COUNT DESCWHERE LOCATION != ' 'LIMIT 5"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSGROUP BY LOCATIONORDER BY COUNT DESCWHERE LOCATION != ''LIMIT 5"
"SELECT COUNT(*) AS COUNT, LOCATIONFROM USERSWHERE LOCATION != ''GROUP BY LOCATIONORDER BY COUNT DESCLIMIT 5"


## Now Your Turn
Find the queries that have 'DISTINCT' in them. You can do it with the %sql command or with Pandas and sql.

<div style="background-color:lemonchiffon; border:1px solid; padding:5px; margin:5px; font-weight:bold"><font color="crimson">
TODO: Complete the following code cell
    </font></div>

In [196]:
%%sql
select query

from queries
where query like '%DISTINCT%'

 * sqlite:///chatdata.db
Done.


query
SELECT COUNT(DISTINCT(LOCATION)) AS COUNTFROM USERS


# Close SQLite

It is good practise to close all relational databases as soon as you are finished updating them.

In [197]:
con.close()

# All Done!

Great job. You now have a good idea for how to use sql and pandas with sql. You can create your own databases from csv files and you can do extensive querying using sql. These are valuable skills that will take you a long ways in todays technological world.